In [10]:
import urllib.request
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
import pandas as pd
import kss

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os

class newsCrawler:
    def __init__(self):
        self.titleList=[]
        self.contentsList=[]
        self.imageList=[]
        self.dateList=[]
        
    # 네이버 뉴스홈
    def mainCrawl(self):    
        # 정치=100 경제=101 사회=102 생활/문화=103 세계=104 IT/과학=105
        # 실제 실행에서는 range(100, 106)으로 해야함.
        for category in range(101, 102):
            main_url = "https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1="+str(category)
            driver.get(main_url)
            
            # '헤드라인 더보기' 버튼이 있다면 누르기       
            self.showMore()
            driver.implicitly_wait(3)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')  
            
            # 헤드라인 가져오기
            self.subCrawl(soup, category)
        driver.quit()
    
    # 더보기버튼 클릭
    def showMore(self):
        try:
            while True:
                driver.find_element_by_xpath('//*[@id="main_content"]/div/div[2]/div[2]/div/a').click()
        except exceptions.ElementNotVisibleException as e:
            pass
        except Exception as e:
            pass
    
    # 헤드라인 뉴스 크롤링
    def subCrawl(self, soup, category):
        # 모든 헤드라인 뉴스 저장
        articles = soup.find_all('div', {'class': 'cluster_group _cluster_content'})
        
        for i in range(len(articles)):
            # 각 뉴스 본문에 있는 이미지와 이미지URL를 저장할 리스트
            images=[]
            imagesURL=[]
            
            # 본문 링크
            temp = articles[i].find_all('div', {'class': 'cluster_text'})[0]
            #print(temp.a.text)         
            conURL = temp.a.get('href')
            html2 = session.get(conURL,headers=headers).content
            soup2 = BeautifulSoup(html2, 'html.parser')
            
            # 기사 제목, 내용 추출
            # 기사 요약 부분있으면 본문에서 제외하기
            summary = soup2.find('strong', {'class':'media_end_summary'})
            if summary==None:
                summary=""
            else:
                summary=summary.text
            
            # 1. 본문 / 2. 제목
            contents = soup2.find('div', id= "articleBodyContents").text.replace("\n"," ").replace(str(summary),"").replace("\t"," ").replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}"," ")
            title=soup2.find('h3',id="articleTitle").text
            
            # 기사 본문이 10문장이하라면 저장하지 않는다.
            if(len(kss.split_sentences(contents)) <= 10):
                continue;
            
            # 3. 날짜
            date=soup2.find('span', {'class','t11'}).text
            self.dateList.append(date)
            
            images=soup2.find_all('span', {'class','end_photo_org'})
            # 5. 이미지url
            for i in range(len(images)):
                imagesURL.append(images[i].find("img")["src"])
                #print(imagesURL[i])
            
            # 기사 제목과 내용, 이미지URL 각 리스트에 저장
            self.titleList.append(title)
            self.contentsList.append(contents)
            self.imageList.append(imagesURL)   
        
#         sum_List=[]
#         for i in range(len(self.titleList)):
#             content_sum = TextRank(self.contentsList[i])
#             content_sum = content_sum.summarize(7)
#             sum_List.append(content_sum)
#         print(content_sum)
        
#         news_sum_df = pd.DataFrame(content_sum, columns=self.titleList)
#         print(news_sum_df)
        
        
        # 엑셀 파일이 없다면 생성하기
        # try catch문으로 가능
        while not os.path.isfile('news.xlsx'):
            temp = pd.DataFrame(columns=('date','title','content','category'))
            temp.set_index('date', inplace=True)
            print(temp)
            temp.to_excel('news.xlsx')
        
        # 엑셀 파일 읽기
        news_df = pd.read_excel('news.xlsx')
        news_df.set_index('date', inplace = True)
        
        # 크롤링한 기사 정보로 데이터프레임 생성
        crawl_df = pd.DataFrame({'title':self.titleList, 'content':self.contentsList, 'date':self.dateList})
        crawl_df['category'] = pd.Series([category for i in range(len(crawl_df.index))])
        crawl_df.set_index('date', inplace = True)
        
        news_df = pd.concat([news_df, crawl_df])
        
        # 엑셀 파일 쓰기
        #news_df.to_excel('news.xlsx')
        
        # 파일 저장
        #self.saveNewsToFile()
        
        # db에 저장
        #self.saveToDB()
        
        #self.cosine_sim(news_df)
    
    def saveNewsToFile(self):
        f = open('result0319.txt', 'w', encoding='utf-8')
        for i in range(len(self.titleList)):
            f.write(self.titleList[i] + "\n")
            f.writelines(self.contentsList[i]+"\n")
            f.writelines(self.imageList[i])
            f.writelines("\n\n")
        f.close()
    
    def saveToDB(self):

#         # SQL문 실행
#         sql = "USE TEST1"
#         curs.execute(sql)
#         '''
#         CREATE TABLE NEWS4 (TITLE CHAR(200) NOT NULL,
#         CONTENT1 TEXT NOT NULL,
#         CONTENT2 TEXT NOT NULL,
#         CONTENT3 TEXT NOT NULL,
#         CONTENT4 TEXT NOT NULL,
#         CONTENT5 TEXT NOT NULL,
#         CONTENT6 TEXT NOT NULL,
#         CONTENT7 TEXT NOT NULL,
#         DATE VARCHAR(40) NOT NULL,
#         CATEGORY VARCHAR(40),
#         COUNT int NOT NULL AUTO_INCREMENT,
#         IMAGE TEXT NOT NULL,
#         CONSTRAINT PLAYER_PK PRIMARY KEY (COUNT));
#         '''

#         #sql3="insert into NEWS3(title,content,date,category,image) VALUES(" +title+ ',' +content+ ',' +date+ ',' +category+ ',' +imagesURL+ ");"
#         sql3="""insert into NEWS7(title,content1,content2,content3,content4,content5,content6,content7,date,category,image) VALUES('%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s');"""%(title,content[0],content[1],content[2],content[3],content[4],content[5],content[6],date,category,imagesURL)
#         curs.execute(sql3)
#         conn.commit()
#         print("db updated!")

        return 1
    
def cosine_sim(count):
    
    
    df =  pd.read_excel('news.xlsx')
    df = df.drop_duplicates(['title'], keep='first', ignore_index=True)
    # 모든 단어들의 빈도에 대하여 유사도를 계산하면 값이 너무 작게 나와서
    # max_features옵션으로 Tf-Idf의 크기를 줄인 다음 코사인 유사도를 계산함.
    tfidf = TfidfVectorizer(max_features=500)
    
    # max_feature는 tf-idf vector의 최대 feature를 설정해주는 파라미터입니다.
    # 머신러닝에서 feature란, 테이블의 컬럼에 해당하는 개념입니다. 또한 행렬의 열에 해당하는 것이기도 합니다.
    # TF-IDF 벡터는 단어사전의 인덱스만큼 feature를 부여받습니다.
    
    # doc: 기사 본문(문서)
    # tfidf_mat: 문서들을 벡터화한 
    doc = list(df['content'])
    tfidf_mat = tfidf.fit_transform(doc).toarray()
    #print(type(tfidf_mat))
    print(tfidf_mat)
    print(tfidf_mat.shape)
    
    # 소수점 4자리까지 반올림
    sim = np.round(cosine_similarity(tfidf_mat, tfidf_mat),4)
    print(sim)
    #print(type(sim))
    print(sim.shape)
    
    #print(sim)
    
    sim_scores = list(enumerate(sim[count]))
    #print(sim_scores)
    
    # 유사도가 높은 순서대로 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 상위 인덱스와 유사도 추출
    sim_scores = sim_scores[1:6]
    print(sim_scores)
    
    # 원하는 기사와 유사한 기사 인덱스를 이용하여 제목 출력
    movie_indices = [i[0] for i in sim_scores]
    result = df['title'].iloc[movie_indices]
    print(result)
    print(tuple(result.index))
    
    df.set_index('date', inplace = True)
    df.to_excel('news.xlsx')
    
    return tuple(result.index)
    
        
        

session = requests.Session()

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
}

#driver = webdriver.Chrome(r"C:\Users\LCM\Downloads\chromedriver_win32\chromedriver.exe")
#driver = webdriver.Chrome(r"C:\Users\seenw\Downloads\chromedriver_win32\chromedriver.exe")

#c=newsCrawler()
#c.mainCrawl()
cosine_sim(6)

[[0.         0.         0.         ... 0.13311031 0.06655516 0.06655516]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
(69, 500)
[[1.     0.2444 0.0766 ... 0.0349 0.0578 0.0618]
 [0.2444 1.     0.0955 ... 0.0569 0.0963 0.105 ]
 [0.0766 0.0955 1.     ... 0.0328 0.0835 0.1081]
 ...
 [0.0349 0.0569 0.0328 ... 1.     0.0727 0.0105]
 [0.0578 0.0963 0.0835 ... 0.0727 1.     0.0266]
 [0.0618 0.105  0.1081 ... 0.0105 0.0266 1.    ]]
(69, 69)
[(8, 0.3603), (2, 0.3527), (48, 0.3384), (43, 0.2622), (3, 0.2333)]
8     [D-1]與 "지지층 결집 박빙 승부" vs 野 "투표율 50%  두자릿수 낙승"
2                     김종인, 모레 국민의힘 떠난다…'주호영 대행' 체제로
48                김종인, 모레 국민의힘 떠난다…'주호영 대행' 체제로(종합)
43           

(8, 2, 48, 43, 3)

In [2]:
from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

import kss


class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.okt = Okt()
        # 불용어 불러오기
        self.stopwords = [line.rstrip('\n') for line in open('stopwords_korean3.txt', encoding = 'utf-8')]
    
    def text2sentences(self, text):
        #sentences = self.kkma.sentences(text)\
        sentences = kss.split_sentences(text)
        # 기사의 마지막 문장 1개 제외시킴
        sentences = sentences[0:len(sentences)-2]
        
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
                
        return sentences
    
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence != '':
                nouns.append(' '.join([noun for noun in self.okt.nouns(str(sentence))
                    if noun not in self.stopwords and len(noun) > 1]))
                
        return nouns


class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
        
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        # tfidf_mat의 크기는 문장수x단어수
#         print("tf-idf 행렬:")
#         print(tfidf_mat)
#         print(tfidf_mat.shape)
        
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
#         print("graph_sentence 행렬:")
#         print(self.graph_sentence)
#         print(self.graph_sentence.shape)
        # graph_sentence는 각 문장 벡터들간의 유사도 행렬이므로
        # 행렬의 크기는 문장수x문장수
        # 그래프로 그리면 노드가 문장수만큼 존재하고,
        # 이 그래프를 textrank 알고리즘의 입력으로 사용하여 각 문장의 점수를 구한다.
        # 점수가 높은 상위 n개를 선택하여 문서의 요약문으로 삼는다.
        
        
        #print(self.tfidf.get_feature_names())
        return self.graph_sentence
    
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}
    
    
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
                
            A[:, id] *= -d
            A[id, id] = 1
            
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

    
    
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences = self.sent_tokenize.text2sentences(text)
            
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        #print(self.sent_graph)
        #print(self.sent_graph.shape)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
        #print(self.nouns)
        
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        
        return summary
    
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
            
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
            
        return keywords

# db와 연결 - db에 있는 기사 본문?요약?들을 tf-idf행렬로 벡터화 - 각 유사도 계산
# - 새로 크롤링한 기사들과 유사한 '최근' 기사 5개씩 db에 저장

# 아니면 db와 연결하지 말고 txt파일로 저장하는걸 추가로 구현한 다음에
# txt를 읽고 유사도를 계산하는게 더 나을수도

# 그럼 txt파일 or csv파일에 어떤 내용들을 저장할까
# 지금까지 크롤링한 기사본문/카테고리(100~105)/tf-idf행렬

# class Recommendation(object):
#     def __init__(self, text):
        
#     def cosine_sim(self):
        
#     def get_recommendation(self):
        

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
import pandas as pd

df = pd.read_excel('news.xlsx')
print(df)

print(type(df))

   Unnamed: 0                               title  \
0           0     바이비트, 6개월만에 비트코인 선물거래소 세계 2위 등극   
1           1   '美금리 부담' 코스피 1% 하락 3030선…개인 홀로 사자   
2           2  LG·SK '배터리 특허침해' 미국 ITC 예비판결 2주 연기   
3           3    삼성물산, 첫 사외이사 이사회 의장 선임…ESG 경영 강화   
4           4   '저세상 매운맛' 눈 뒤집힌 해외…불닭볶음면 3000억 대박   
5           5              그린북서 ‘불확실성’ 표현 삭제한 기재부   

                                             content                  date  \
0             비트코인 선물거래소 순위에서 6위에 머물던 바이비트가 불과 6개...   2021.03.18. 오후 3:48   
1          15일 오후 서울 중구 하나은행 딜링룸 전광판. 2021.3.15/뉴...   2021.03.19. 오전 9:52   
2          LG에너지솔루션이 SK이노베이션을 상대로 미국 국제무역위원회에 제기한...   2021.03.19. 오전 9:31   
3          뉴스1 © News1 구윤성 기자(서울=뉴스1) 이훈철 기자 = 정병...   2021.03.19. 오후 2:23   
4          삼양식품이 매운맛으로 특히 동남아에서 인기가 높은 ‘불닭볶음면’을 앞...  2021.03.19. 오전 10:37   
5          정부가 경기평가에서 10개월만에 ‘불확실성’이라는 표현을 삭제했다. ...  2021.03.19. 오전 11:39   

   category  
0       101  
1       101  
2       101  
3       101  
4       101

In [13]:
s = 'sss'

def f(s):
    return '안녕'+s

print(f(s))

안녕sss


In [23]:
text='''“연구자들이 성과를 보여주기 위해 자잘한 논문을 내는 경우가 많습니다. 기초과학연구원(IBS) 소속이 아니면 연구비 받기도 힘들다는 얘기도 나옵니다.”(유룡 KAIST 화학과 교수)올해로 일본이 3년 연속 노벨생리의학상 등 그동안 총 22명의 노벨과학상 수상자를 배출하는 사이 한국은 116년 노벨상 역사상 과학 분야에서 한 명의 수상자도 배출하지 못했다. 중국도 지난해 처음으로 노벨생리의학상을 공동 수상했다. 연구개발(R&D) 투자 비중이 국내총생산(GDP) 대비 4.29%(86조원)로 세계 최고 수준인 우리나라로서는 근본적인 대책을 세워야 한다는 지적이 나온다.
연구 일선에서는 정부가 당장 돈벌이가 되는 분야에만 몰리고 기초과학 연구에 상대적으로 소홀하다는 비판이 나온다. 연구예산이 증가해도 반도체·통신 등 정보기술(IT) 분야에만 매달릴 뿐 응용·첨단기술의 토양인 기초과학은 뒷전이라는 것이다. 지난 3월 인공지능(AI) 알파고가 주목받자 정부가 ‘한국형 알파고’ 계획을 내놓은 것이 단적인 예다. 유룡 교수는 “기초과학 투자를 늘리지 않고 노벨상을 바라는 것은 이미 20년 전 나온 AI 기술을 미리 키우지도 않고 당장 하겠다는 것과 같다”고 지적했다. 한국과학기술기획평가원에 따르면 2015년 우리나라의 R&D 예산 18조8,900억원 중 기초과학 예산은 1조2,081억원으로 6.4%에 불과하다.
지난해 대통령 주재 과학기술자문회의에서 노벨상을 받을 만한 과학자 1,000명을 육성하겠다는 계획을 발표했으나 우선 인프라에 투자해야 한다는 목소리도 있다. 김선영 서울대 생명과학부 교수는 “기초과학계는 김연아의 후계자를 꿈꾸는 피겨스케이터들이 제대로 된 빙상장조차 없이 훈련하는 것과 같은 열악한 수준에서 실험을 하고 있다”고 비유했다.
연구과제를 수주하기 위해 투서 등 과도한 경쟁과 공무원이 절대적인 영향력을 행사하며 공정성 시비가 끊이지 않는 연구예산 배분 과정도 개선돼야 한다는 지적이다. 박배호 건국대 물리학과 교수는 “과도한 경쟁과 줄서기가 요구되는 상황에서 창의적이고 분야를 이끄는 연구를 수행하기 어렵다”며 “연구자들이 적절한 평가 및 선정 과정을 거쳐 다른 곳에 신경 쓸 필요없이 연구를 수행할 수 있는 생태계가 조성돼야 한다”고 강조했다. 이 같은 분위기가 만들어져야 과학고등학교 학생들이 의과대로 몰리고 과학기술특성화대 졸업자들이 로스쿨과 의대로 떠나는 것을 막을 수 있다. 시민단체 ‘사교육 걱정없는 세상’ 등에 따르면 지난 2011~2015학년도 5년간 서울과학고에서 의학계열로 진학한 비율은 20.5%로 졸업생 5명 중 1명에 이른다.
과학기술을 담당하는 미래창조과학부는 정보통신기술(ICT) 쪽으로 기운 인사를 하고 있다는 비판이 제기된다. 김경진 국회 미래창조과학방송통신위원(국민의당)은 최근 국정감사에서 2013년 3월 미래부 출범 시 실·국장급 이상 고위공무원 28명 중 과학기술부 출신은 11명(39%), ICT 출신 9명(32%), 기획재정부·지식경제부 외 7명(1명 공석)이었지만 2016년 현재 고위공무원 27명 중 과기부 출신은 8명(29%)으로 3명이 줄었고 ICT 출신은 4명이 늘어 13명(48%)이 됐다고 지적했다.
1962년 이래 13명의 노벨 수상자를 배출한 영국 분자생물학실험실(LMB)이 연예산을 600억원 정도 쓰면서 시너지를 내는 것은 창의적인 교육을 받은 학생들이 맘 놓고 연구할 수 있는 기초과학 인프라를 구축했기 때문이라는 분석도 나온다. 김선영 교수는 “듬직한 나이의 중견 과학자들은 좌고우면할 것이 많고 진행 중인 과제의 동력에 함몰돼 결과에 충격과 감동이 없는 경우가 대부분”이라며 “젊은이들은 번뜩이는 아이디어뿐 아니라 모험적인 창의적 과제에 도전할 수 있는 용기와 배짱을 가지고 있다”고 말했다.
윤종용 한국공학교육인증원 이사장(전 삼성전자 부회장)은 최근 서울경제신문과의 인터뷰에서 “우리나라 연구조직을 보면 여전히 연공서열과 학연을 따지고 연구 활동보다 정부의 프로젝트를 따기 위한 문서작업이나 공무원 응대에 더 많은 시간을 쏟는 것 같다”며 “단기성과 위주의 정량 평가는 연구자들의 자율성을 떨어뜨리고 사기를 저하시킨다”고 지적했다.'''
count=0
for sent in kss.split_sentences(text):
    count+=1
    print(count)
    print(sent)
    print()
    
# summary=[]
# sum = TextRank(text)
#         #print(sum.summarize(8)) # 길이가 5인 리스트 출력
#         #print(len(sum.summarize(8)))
# for row in sum.summarize(6):
#     summary.append(row)
#     print(row)
#     print()



1
“연구자들이 성과를 보여주기 위해 자잘한 논문을 내는 경우가 많습니다. 기초과학연구원(IBS) 소속이 아니면 연구비 받기도 힘들다는 얘기도 나옵니다.”(유룡 KAIST 화학과 교수) 올해로 일본이 3년 연속 노벨생리의학상 등 그동안 총 22명의 노벨과학상 수상자를 배출하는 사이 한국은 116년 노벨상 역사상 과학 분야에서 한 명의 수상자도 배출하지 못했다.

2
중국도 지난해 처음으로 노벨생리의학상을 공동 수상했다.

3
연구개발(R&D) 투자 비중이 국내총생산(GDP) 대비 4.29%(86조원)로 세계 최고 수준인 우리나라로서는 근본적인 대책을 세워야 한다는 지적이 나온다.

4
연구 일선에서는 정부가 당장 돈벌이가 되는 분야에만 몰리고 기초과학 연구에 상대적으로 소홀하다는 비판이 나온다.

5
연구예산이 증가해도 반도체·통신 등 정보기술(IT) 분야에만 매달릴 뿐 응용·첨단기술의 토양인 기초과학은 뒷전이라는 것이다.

6
지난 3월 인공지능(AI) 알파고가 주목받자 정부가 ‘한국형 알파고’ 계획을 내놓은 것이 단적인 예다.

7
유룡 교수는 “기초과학 투자를 늘리지 않고 노벨상을 바라는 것은 이미 20년 전 나온 AI 기술을 미리 키우지도 않고 당장 하겠다는 것과 같다”고 지적했다. 한국과학기술기획평가원에 따르면 2015년 우리나라의 R&D 예산 18조8,900억원 중 기초과학 예산은 1조2,081억원으로 6.4%에 불과하다.

8
지난해 대통령 주재 과학기술자문회의에서 노벨상을 받을 만한 과학자 1,000명을 육성하겠다는 계획을 발표했으나 우선 인프라에 투자해야 한다는 목소리도 있다.

9
김선영 서울대 생명과학부 교수는 “기초과학계는 김연아의 후계자를 꿈꾸는 피겨스케이터들이 제대로 된 빙상장조차 없이 훈련하는 것과 같은 열악한 수준에서 실험을 하고 있다”고 비유했다.

10
연구과제를 수주하기 위해 투서 등 과도한 경쟁과 공무원이 절대적인 영향력을 행사하며 공정성 시비가 끊이지 않는 연구예산 배분 과정도 개선돼야 한다는 지적이다.

11
박배호 건국